In [1]:
import sys, os
CURRENT_TEST_DIR = os.getcwd()
sys.path.append(CURRENT_TEST_DIR + "/../../src")

In [2]:
import sys
print(sys.version)

3.7.6 (default, Jan  8 2020, 19:59:22) 
[GCC 7.3.0]


In [3]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import slayerSNN as snn
from learningStats import learningStats
from IPython.display import HTML
import zipfile

In [4]:
netParams = snn.params('network_grid.yaml')

In [5]:
# Dataset definition
class nmnistDataset(Dataset):
    def __init__(self, datasetPath, sampleFile, samplingTime, sampleLength):
        self.path = datasetPath 
        self.samples = np.loadtxt(sampleFile).astype('int')
        self.samplingTime = samplingTime
        self.nTimeBins    = int(sampleLength / samplingTime)

    def __getitem__(self, index):
        inputIndex  = self.samples[index, 0]
        classLabel  = self.samples[index, 1]

        inputSpikes = snn.io.readNpSpikes(
                        self.path + str(inputIndex.item()) + '.npy'
                        ).toSpikeTensor(torch.zeros((1,300,200,self.nTimeBins)),
                        samplingTime=self.samplingTime)
        desiredClass = torch.zeros((30, 1, 1, 1))
        desiredClass[classLabel,...] = 1
        return inputSpikes, desiredClass, classLabel

    def __len__(self):
        return self.samples.shape[0]

In [6]:
class Network(torch.nn.Module):
    def __init__(self, netParams):
        super(Network, self).__init__()
        # initialize slayer
        slayer = snn.layer(netParams['neuron'], netParams['simulation'])
        self.slayer = slayer
        # define network functions
        self.conv1 = slayer.conv(1, 8, 4, stride=2, padding=1)
        self.conv2 = slayer.conv(8, 16, 4, stride=2, padding=1)
        self.conv3 = slayer.conv(16, 32, 4, padding=1)
        self.pool1 = slayer.pool(2)
        self.pool2 = slayer.pool(2)
        self.fc1   = slayer.dense((12, 18, 32), 30)

    def forward(self, spikeInput):
        spikeLayer1 = self.slayer.spike(self.conv1(self.slayer.psp(spikeInput ))) # 32, 32, 16
        spikeLayer2 = self.slayer.spike(self.pool1(self.slayer.psp(spikeLayer1))) # 16, 16, 16
        spikeLayer3 = self.slayer.spike(self.conv2(self.slayer.psp(spikeLayer2))) # 16, 16, 32
        spikeLayer4 = self.slayer.spike(self.pool2(self.slayer.psp(spikeLayer3))) #  8,  8, 32
        spikeLayer5 = self.slayer.spike(self.conv3(self.slayer.psp(spikeLayer4))) #  8,  8, 64
        spikeOut    = self.slayer.spike(self.fc1  (self.slayer.psp(spikeLayer5))) #  10

        return spikeOut

In [7]:
# Define the cuda device to run the code on.
# device = torch.device('cuda')
# Use multiple GPU's if available
device = torch.device('cuda:0') # should be the first GPU of deviceIDs
deviceIds = [0, 1]

# Create network instance.
# net = Network(netParams).to(device)
# Split the network to run over multiple GPUs
net = torch.nn.DataParallel(Network(netParams).to(device), device_ids=deviceIds)

# Create snn loss instance.
error = snn.loss(netParams).to(device)

# Define optimizer module.
optimizer = torch.optim.Adam(net.parameters(), lr = 0.01, amsgrad = True)
    
# Dataset and dataLoader instances.
trainingSet = nmnistDataset(datasetPath =netParams['training']['path']['in'], 
                            sampleFile  =netParams['training']['path']['train'],
                            samplingTime=netParams['simulation']['Ts'],
                            sampleLength=netParams['simulation']['tSample'])
trainLoader = DataLoader(dataset=trainingSet, batch_size=8, shuffle=True, num_workers=2)

testingSet = nmnistDataset(datasetPath  =netParams['training']['path']['in'], 
                            sampleFile  =netParams['training']['path']['test'],
                            samplingTime=netParams['simulation']['Ts'],
                            sampleLength=netParams['simulation']['tSample'])
testLoader = DataLoader(dataset=testingSet, batch_size=8, shuffle=True, num_workers=2)

# Learning stats instance.
stats = learningStats()

## Visualize

In [8]:
input, target, label = trainingSet[102]
print(target)
print(label)
print(input.shape)
anim = snn.io.animTD(snn.io.spikeArrayToEvent(input.reshape((1, 300, 200, -1)).cpu().data.numpy()))
HTML(anim.to_jshtml())

tensor([[[[0.]]],


        [[[1.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]],


        [[[0.]]]])
1
torch.Size([1, 300, 200, 100])


In [9]:
neighbor_file = open("../dataset/KH/clean/raw_numpy_250ms/2020_07_04_11_08_58_neighbor_40.txt",'r')
neighbor_dict = dict()
while True:
    line = neighbor_file.readline()
    if not line:
        break
    sp_line = line.split()
    neighbor_dict[int(sp_line[0])] = set()
    for i in range(1,len(sp_line)):
        neighbor_dict[int(sp_line[0])].add(int(sp_line[i]))
print(neighbor_dict)
neighbor_file.close()

{0: {1, 4, 5, 6}, 1: {0, 2, 3, 4, 5}, 2: {1, 3, 4}, 3: {1, 2, 4}, 4: {0, 1, 2, 3, 5}, 5: {0, 1, 4, 6}, 6: {0, 8, 5, 7}, 7: {8, 6}, 8: {9, 6, 7}, 9: {8, 10, 11}, 10: {9, 11}, 11: {9, 10, 12, 13, 29}, 12: {13, 11, 29, 14}, 13: {11, 12, 14}, 14: {12, 13, 15}, 15: {16, 14}, 16: {17, 18, 15}, 17: {16, 18}, 18: {16, 17, 19, 20}, 19: {18, 20, 21}, 20: {18, 19, 21}, 21: {19, 20, 22, 23}, 22: {21, 23}, 23: {24, 21, 22}, 24: {25, 23}, 25: {24, 26, 27}, 26: {25, 27, 28}, 27: {25, 26, 28, 29}, 28: {26, 27, 29}, 29: {11, 27, 12, 28}}


# Train the network

In [10]:
for epoch in range(100):
    # Reset training stats.
    stats.training.reset()
    tSt = datetime.now()

    # Training loop.
    for i, (input, target, label) in enumerate(trainLoader, 0):
        # Move the input and target to correct GPU.
        input  = input.to(device)
        #print(input)
        target = target.to(device) 

        # Forward pass of the network.
        output = net.forward(input)

        # Gather the training stats.
        result = snn.predict.getClass(output)
        result = result.tolist()
        label = label.tolist()
        
        count = 0
        for i in range(len(label)):
            if (label[i] == result[i]) or (label[i] in neighbor_dict[result[i]]):
                count += 1
        stats.training.correctSamples += count
        stats.training.numSamples     += len(label)

        # Calculate loss.
        loss = error.numSpikes(output, target)

        # Reset gradients to zero.
        optimizer.zero_grad()

        # Backward pass of the network.
        loss.backward()

        # Update weights.
        optimizer.step()

        # Gather training loss stats.
        stats.training.lossSum += loss.cpu().data.item()

        # Display training stats. (Suitable for normal python implementation)
        # stats.print(epoch, i, (datetime.now() - tSt).total_seconds())

    # Update training stats.
    stats.training.update()
    # Reset testing stats.
    stats.testing.reset()

    # Testing loop.
    # Same steps as Training loops except loss backpropagation and weight update.
    for i, (input, target, label) in enumerate(testLoader, 0):
        input  = input.to(device)
        target = target.to(device) 

        output = net.forward(input)
        
        result = snn.predict.getClass(output)
        result = result.tolist()
        label = label.tolist()
        
        count = 0
        for i in range(len(label)):
            if (label[i] == result[i]) or (label[i] in neighbor_dict[result[i]]):
                count += 1

        stats.testing.correctSamples += count
        stats.testing.numSamples     += len(label)

        loss = error.numSpikes(output, target)
        stats.testing.lossSum += loss.cpu().data.item()
        # stats.print(epoch, i)

    # Update testing stats.
    stats.testing.update()
    if epoch%1==0:  stats.print(epoch, timeElapsed=(datetime.now() - tSt).total_seconds())


Epoch :          0, 1430647.2170 ms elapsed
loss = 28.45       (min = 28.45      )    accuracy = 0.62466  (max = 0.62466 )
loss = 19.186      (min = 19.186     )    accuracy = 0.40183  (max = 0.40183 )

Epoch :          1, 1447053.6020 ms elapsed
loss = 8.2335      (min = 8.2335     )    accuracy = 0.87943  (max = 0.87943 )
loss = 18.799      (min = 18.799     )    accuracy = 0.37857  (max = 0.40183 )

Epoch :          2, 1453071.0340 ms elapsed
loss = 5.9655      (min = 5.9655     )    accuracy = 0.93586  (max = 0.93586 )
loss = 21.448      (min = 18.799     )    accuracy = 0.38387  (max = 0.40183 )

Epoch :          3, 1446614.1660 ms elapsed
loss = 4.7353      (min = 4.7353     )    accuracy = 0.95548  (max = 0.95548 )
loss = 23.771      (min = 18.799     )    accuracy = 0.36473  (max = 0.40183 )

Epoch :          4, 1448196.4150 ms elapsed
loss = 3.8685      (min = 3.8685     )    accuracy = 0.96953  (max = 0.96953 )
loss = 19.725      (min = 18.799     )    accuracy = 0.38034  (m

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f7010c7d050>
Traceback (most recent call last):
  File "/home/student1/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1101, in __del__
    self._shutdown_workers()
  File "/home/student1/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1075, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/student1/anaconda3/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/home/student1/anaconda3/lib/python3.7/multiprocessing/popen_fork.py", line 45, in wait
    if not wait([self.sentinel], timeout):
  File "/home/student1/anaconda3/lib/python3.7/multiprocessing/connection.py", line 920, in wait
    ready = selector.select(timeout)
  File "/home/student1/anaconda3/lib/python3.7/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt:

Traceback (most recent call last):
  File "/home/student1/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-055468e67981>", line 41, in <module>
    stats.training.lossSum += loss.cpu().data.item()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/student1/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/student1/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1151, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/st

KeyboardInterrupt: 

In [ ]:
#for i, (input, target, label) in enumerate(testLoader, 0):
#        input  = input.to(device)
#        target = target.to(device) 

#        output = net.forward(input)
#        result = snn.predict.getClass(output)
#        print(result,label)

In [ ]:
testSet = nmnistDataset(datasetPath  ="/home/student1/hyeongilee/slayerPytorch/example/dataset/KH/clean/raw_numpy_250ms/test_set/", 
                            sampleFile  ="/home/student1/hyeongilee/slayerPytorch/example/dataset/KH/clean/raw_numpy_250ms/test_set/test.txt",
                            samplingTime=1,
                            sampleLength=250)
Loader = DataLoader(dataset=testingSet, batch_size=1, shuffle=False, num_workers=2)
#print(testSet[0][0])

for i, (input, target, label) in enumerate(Loader, 0):
        # Move the input and target to correct GPU.
        input  = input.to(device)
        target = target.to(device) 

        output = net.forward(input)
        result = snn.predict.getClass(output)
        print(result)
#for i in range(1008):
#    f = np.load(/dataset/KH/clean/raw_numpy_250ms/test_set)

## Plot

In [ ]:
plt.figure(1)
plt.semilogy(stats.training.lossLog, label='Training')
plt.semilogy(stats.testing .lossLog, label='Testing')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.figure(2)
plt.plot(stats.training.accuracyLog, label='Training')
plt.plot(stats.testing .accuracyLog, label='Testing')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()